### Initialize 


In [1]:
import csv
import json
import praw
import pandas as pd
from datetime import datetime, timezone

client_id = 'XqvMFAQqrS1xWiUpCcbSOg'
client_secret = '1WDnClQ3eo8Luzug2FSmZ6x9Aj1Ytg'
user_agent = 'ios:MyRedditApp:v1.0 (by /u/sk_shuanq)'

reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

# Test if connected
print(reddit.read_only)  # Expected True

try:
    # Test with subreddit: python
    subreddit = reddit.subreddit('python')
    print(f"Successfully connected! Subreddit title: {subreddit.title}")
    print(f"Read-only mode: {reddit.read_only}")  # True Expected
except Exception as e:
    print(f"Connection failed: {e}")

True
Successfully connected! Subreddit title: Python
Read-only mode: True


## 2. Text (Submission&Comments) Collection

In [2]:
file_path_subreddit = "data/processed-data/subreddits.csv"
df_subreddit = pd.read_csv(file_path_subreddit)
subreddit_list = df_subreddit['subreddit'].tolist()

# Storage for results
results = []

# Counters for submission and comments
submission_count = 0
comment_count = 0

# Function to convert UTC timestamp to readable format
def convert_timestamp(utc_timestamp):
    return datetime.fromtimestamp(utc_timestamp, timezone.utc).strftime('%Y/%m')

# Iterate over each subreddit
for subreddit_name in subreddit_list:
    print(f"Fetching data for subreddit: {subreddit_name}")
    try:
        subreddit = reddit.subreddit(subreddit_name)
        
        # Store post IDs to avoid duplicates
        seen_post_ids = set()
        
        # Submission Collection Part 1: Get top 10 posts by hot ranking
        for submission in subreddit.hot(limit=10):
            
            # Skip repeat submission
            if submission.id in seen_post_ids:
                continue

            # Skip posts with the "meme" flair
            if submission.link_flair_text and "meme" in submission.link_flair_text.lower():
                print(f"Skipping post with 'meme' flair: {submission.title}")
                continue

            submission_data = {
                "subreddit": subreddit_name,
                "post_id": submission.id,
                "title": submission.title,
                "text": submission.selftext,
                "score": submission.score,
                "created_utc": convert_timestamp(submission.created_utc),
                "sort_type": "hot",
                "comments": []
            }

            seen_post_ids.add(submission.id)
            submission_count += 1

            # Process comments for this submission
            submission.comments.replace_more(limit=0)  
            all_comments = submission.comments.list()
            
            # Sort comments by number of replies
            comments_by_replies = sorted(
                all_comments,
                key=lambda x: len(x.replies) if hasattr(x, 'replies') else 0,
                reverse=True
            )[:10]
            
            # Process comments and add indentation
            def process_comments(comments, depth=0):
                global comment_count
                processed_comments = []
                
                for comment in comments:
                    # indent = "  " * depth  
                    processed_comment = {
                        "comment_id": comment.id,
                        "body": comment.body,
                        "score": comment.score,
                        "depth": depth,
                        "num_replies": len(comment.replies) if hasattr(comment, 'replies') else 0,
                        "created_utc": convert_timestamp(comment.created_utc)  
                    }
                    
                    comment_count += 1
                    processed_comments.append(processed_comment)
                    
                    # Process replies if they exist
                    if hasattr(comment, 'replies') and len(comment.replies) > 0:
                        replies = list(comment.replies)[:10]  # Limit replies to 10 per comment
                        processed_comments.extend(process_comments(replies, depth + 1))
                
                return processed_comments

            # Process and store comments for the current submission
            submission_data["comments"] = process_comments(comments_by_replies)
            results.append(submission_data)


        # Submission Collection Part 2: Get top 10 posts by controversial ranking
        for submission in subreddit.controversial(limit=10):
            if submission.id in seen_post_ids:
                continue
                
            # Skip posts with the "meme" flair
            if submission.link_flair_text and "meme" in submission.link_flair_text.lower():
                print(f"Skipping post with 'meme' flair: {submission.title}")
                continue
            
            submission_data = {
                "subreddit": subreddit_name,
                "post_id": submission.id,
                "title": submission.title,
                "text": submission.selftext,
                "score": submission.score,
                "created_utc": convert_timestamp(submission.created_utc),  
                "sort_type": "controversial",  
                "comments": []
            }
            
            seen_post_ids.add(submission.id)
            submission_count += 1

            # Process comments for this submission
            submission.comments.replace_more(limit=0)
            all_comments = submission.comments.list()
            
            # Sort comments by number of replies
            comments_by_replies = sorted(
                all_comments,
                key=lambda x: len(x.replies) if hasattr(x, 'replies') else 0,
                reverse=True
            )[:10]
            
            # Process and store comments using the same process_comments function
            submission_data["comments"] = process_comments(comments_by_replies)
            results.append(submission_data)

    except Exception as e:
        print(f"Error fetching data for subreddit {subreddit_name}: {e}")
        continue

# Print summary of fetched data
print(f"Total posts fetched: {submission_count}")
print(f"Total comments fetched: {comment_count}")

Fetching data for subreddit: Libertarian
Skipping post with 'meme' flair: Statism in a Nutshell
Skipping post with 'meme' flair: Welcome to the Twilight Zone
Skipping post with 'meme' flair: How a Modern Day Mengele; became hugely wealthy promoting the NWO Agenda
Fetching data for subreddit: LibertarianLeft
Fetching data for subreddit: LibertarianSocialism
Fetching data for subreddit: Conservative
Fetching data for subreddit: conservatives
Fetching data for subreddit: conservativeterrorism
Fetching data for subreddit: politics
Fetching data for subreddit: Republican
Fetching data for subreddit: republicans
Fetching data for subreddit: RepublicanValues
Fetching data for subreddit: RepublicanPedophiles
Fetching data for subreddit: democrats
Skipping post with 'meme' flair: Forget "of the people, by the people." America is now of the wealthy, by the corporations.
Skipping post with 'meme' flair: No Man Should View Trump as a Role Model
Fetching data for subreddit: DemocraticSocialism
Fetc

In [3]:
# Save results to a JSON file
file_path_text_raw = "data/raw-data/text_raw.json"
with open(file_path_text_raw, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print(f"Data collection complete. Results saved to {file_path_text_raw}.")


Data collection complete. Results saved to data/raw-data/text_raw.json.


### File Format Converting

In [6]:
# Read the json data
def convert_reddit_data(json_input_path, csv_output_path):
    with open(json_input_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Lists to store the processed data
    processed_data = []

    # Process each submission and its comments
    for submission in data:
        submission_text = submission['title']
        if submission['text']:  # Combine text content if it exists
            submission_text += " " + submission['text']
            
        # Add submission data
        processed_data.append({
            'subreddit': submission['subreddit'],
            'id': submission['post_id'],
            'type': f"submission_{submission['sort_type']}",  # submission_hot or submission_con
            'depth': 0,  # submissions are depth 0
            'score': submission['score'],
            'time': submission['created_utc'],
            'text': submission_text
        })
        
        # Process comments
        for comment in submission['comments']:
            processed_data.append({
                'subreddit': submission['subreddit'],
                'id': comment['comment_id'],
                'type': 'comment',
                'depth': comment['depth'] + 1,  # increment depth by 1
                'score': comment['score'],
                'time': comment['created_utc'],
                'text': comment['body']
            })

    # Convert to DataFrame
    df = pd.DataFrame(processed_data)
    
    # Save to CSV
    df.to_csv(csv_output_path, index=False)
    
    # Print some basic statistics
    print("\nFile convert complete. Results saved to data/processed-data/text_inital.json.")
    print(f"Total rows: {len(df)}")
    print("\nDistribution by type:")
    print(df['type'].value_counts())
    print("\nDepth distribution:")
    print(df['depth'].value_counts().sort_index())

    return df

# Execute the conversion
file_path_text_raw = "data/raw-data/text_raw.json"
file_path_text_initial = "data/processed-data/text_initial.csv"  

df_text_initial = convert_reddit_data(file_path_text_raw, file_path_text_initial)
df_text_initial.head(6)


File convert complete. Results saved to data/processed-data/text_inital.json.
Total rows: 59724

Distribution by type:
type
comment                     59032
submission_hot                347
submission_controversial      345
Name: count, dtype: int64

Depth distribution:
depth
0       692
1      5573
2     13747
3     11078
4      9039
5      6583
6      4939
7      3411
8      2353
9      1495
10      814
Name: count, dtype: int64


,subreddit,id,type,depth,score,time,text
0,Libertarian,1hf706u,submission_hot,0,100,2024/12,The road to serfdom New to libertarian economi...
1,Libertarian,m29svuv,comment,1,1,2024/12,Anything by Fredrich Bastiat is also good but ...
2,Libertarian,m2a5co6,comment,2,1,2024/12,Libertarians aren't exclusively anarchists.
3,Libertarian,m2a694w,comment,3,1,2024/12,True but like socialists and communists always...
4,Libertarian,m2a5co6,comment,1,1,2024/12,Libertarians aren't exclusively anarchists.
5,Libertarian,m2a694w,comment,2,1,2024/12,True but like socialists and communists always...


### Text Cleaning

In [14]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import emoji

# Download NLTK stopwords and WordNet data
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Function to clean text
def clean_text(text):
    # Remove extra spaces
    text = text.strip()
    
    # Remove newline characters and extra spaces
    text = re.sub(r'\s+', ' ', text)  # Replace multiple whitespace characters (including newline, tabs) with a single space
    
    # Remove meaningless characters, such as special symbols 
    text = re.sub(r'[^\w\s,.\-]', '', text)  # Keep letters, numbers, spaces, commas, periods, and hyphens
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)  # Remove links
    text = re.sub(r'@\S+', '', text)  # Remove @username mentions
    text = re.sub(r'#[\w]+', '', text)  # Remove all hashtags (e.g., #aiart)
    text = re.sub(r'-', '', text)  # Remove hyphens
    
    # Remove emojis
    text = emoji.replace_emoji(text, replace='')  
    
    # Convert the text to lowercase
    text = text.lower()

    # Remove double quotes
    text = text.replace('"', '')  

    if text.startswith('"') and text.endswith('"'):
        text = text[1:-1]

    return text

def remove_non_english_words(text):
    # English words only
    english_words = re.findall(r'\b[a-zA-Z]+\b', text)
    return ' '.join(english_words)

def process_text(text):
    # Tokenize the text into words
    words = text.split()
    # Remove stop words and apply lemmatization
    processed_words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    # Join the processed words back into a string
    return ' '.join(processed_words)

# Clean the text of each row in the 'text' column
df_text_initial['text'] = df_text_initial['text'].apply(lambda x: clean_text(x))  # Clean the 'text' field
df_text_initial['text'] = df_text_initial['text'].apply(lambda x: remove_non_english_words(x))  # Keep only English words
df_text_initial['text'] = df_text_initial['text'].apply(lambda x: process_text(x))  # Remove stop words and apply lemmatization

# Drop duplicate rows based on the 'text' column
df_text_clean = df_text_initial.drop_duplicates(subset=['text'])

# Output the cleaned DataFrame to a CSV file
file_path_text_clean = "data/processed-data/text_clean.csv"
df_text_clean.to_csv(file_path_text_clean, index=False, encoding='utf-8')

print(f"Data cleaning complete. Results saved to {file_path_text_clean}")
df_text_clean.head(6)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/guochenxi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/guochenxi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Data cleaning complete. Results saved to data/processed-data/text_clean.csv


,subreddit,id,type,depth,score,time,text
0,Libertarian,1hf706u,submission_hot,0,100,2024/12,road serfdom new libertarian economics dad let...
1,Libertarian,m29svuv,comment,1,1,2024/12,anything fredrich bastiat also good think stat...
2,Libertarian,m2a5co6,comment,2,1,2024/12,libertarian arent exclusively anarchist
3,Libertarian,m2a694w,comment,3,1,2024/12,true like socialist communist always saying fi...
6,Libertarian,m29bopi,comment,1,7,2024/12,yeah good favorite always recommend get starte...
7,Libertarian,m2a1cm0,comment,1,1,2024/12,salma write book becoming movie star
